# IDOC Population Data

## Data Collection
#### - Scrape IDOC page for population report links
#### - Download Files
[Population Data Sets](https://www2.illinois.gov/idoc/reportsandstatistics/Pages/Prison-Population-Data-Sets.aspx)

In [1]:
import re
import urllib.request
import requests
import os

base_folder = 'D:\\IDOC\\'
dest_folder = base_folder + 'raw\\'
processing_folder = base_folder + 'proc\\'
clean_folder = base_folder + 'clean\\'
# separate file for all the config values, magic strings?
base = 'https://www2.illinois.gov'
folder = '/idoc/reportsandstatistics/'
docs = 'Documents/'
pages = 'Pages/'
page = 'Prison-Population-Data-Sets.aspx'
url = base + folder + pages + page
print('url = '  + url)

# processing directories
#TODO: S3 buckets
if not os.path.exists(dest_folder):
    os.mkdir(dest_folder) 
if not os.path.exists(clean_folder):
    os.mkdir(clean_folder)
if not os.path.exists(processing_folder):
    os.mkdir(processing_folder)
    
os.chdir(dest_folder)
print('CWD = ' + os.getcwd())
page_source = urllib.request.urlopen(url).read().decode('utf-8')
regex_pattern = '<a href="' + folder + docs + '(.*?)"'

files = []

doc_links = re.findall(regex_pattern, page_source)
for doc_link in doc_links:
   #Process the data from the page here.
   print('>', end="")
   clean_name = urllib.parse.unquote(doc_link)  
   files.append(clean_name)
   r = requests.get(base + folder + docs + doc_link, allow_redirects=True)
   open(clean_name, 'wb').write(r.content)

print("")

print("Files:", len(files), files)



url = https://www2.illinois.gov/idoc/reportsandstatistics/Pages/Prison-Population-Data-Sets.aspx
CWD = D:\IDOC\raw
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Files: 38 ['March 2021  Prison Stock.xls', 'Dec 2020  Prison Stock.xls', 'Sept 2020 Prison Pop.xls', 'June 2020 Prison Stock Pop.xls', 'March 2020 Prison Stock.xls', 'Dec 31 2019 Prison Stock pop.xls', 'September 2019 Prison Stock.xls', 'June 2019 Prison Stock.xls', 'March 2019 Prison Stock.xls', 'December 2018_ Prison Stock.xls', 'September 2018 _Prison_Stock.xls', 'June 2018 Prison Internet Data Set.xls', 'March 2018 Prison_Stock_Internet.xls', 'Internet Data Set Prison_Stock_Dec_2017.xls', 'Internet Data Set Prison_Stock_June 2017.xls', 'Internet Data Set Prison_Stock_Dec_2016.xls', 'Prison Population 6-30-16 Data Set.xls', 'Internet Data Set Prison_Stock_Dec_2015.xls', 'June 2015_ Prison_Stock_Internet_Variables.xls', 'Dec 2014_ Prison_Stock_Internet_Variables.xls', 'June 2014_ Prison_Stock_Internet_Variables.xls', 'Dec 2013_Priso

## Data Preparation

##### Rename dataset files based on month/year values
##### convert files to CSV, move to processing directory

In [2]:
import pandas as pd
import os
  
'''  rename file, convert to csv'''
def process_file(filename):
    new_filename = (processing_folder + normalize_filename(filename) + '.csv')
    pd.read_excel(filename).to_csv (new_filename, 
                  index = None,
                  header=True)

''' find the month, year values in filename
    return in sortable format: monthyear.csv e.g 201012.csv'''
def normalize_filename(filename): 
    new_filename = filename
    
    month_pattern = '(?:Mar(?:ch)?|Jun(?:e)?|Sept(?:ember)?|Dec(?:ember)?)'
    year_pattern = '(?:20[0-1]\d|2\d{3})'
    # 6-30-16 <-- format needed
    alt_date_pattern = '(?:([1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])[- /.](\d\d))'
    
    year = None
    try:
        month = re.findall(month_pattern, filename)
    except Exception as e:        
        print(">>>>>>>>>>>>>>>", e)        
        
    if len(month) == 0:        
        alt_date = re.findall(alt_date_pattern, filename)
        if len(alt_date) > 0: 
            month = alt_date[0][0]
            if len(month) == 1:       
                month = "0" + month                
            year = '20' + alt_date[0][2]
    else:         
        month = mapMonth(month[0])            
 
    if year is None:
        year = re.findall(year_pattern, filename)[0]
    
    #print("Month:", month,"Year:",year)
    return (year + month)

''' map the various month strings used to appropriate month string'''
#TODO: find the module that does this already
def mapMonth(month_str):
    month_str_to_int_dict = {'March':'03', 'June':'06', 'September':'09', 'Sept':'09', 'Dec':'12', 'December':'12'}
    return month_str_to_int_dict[month_str]
    
    
os.chdir(dest_folder)
    
print('Processing Files in ', dest_folder)
for file in os.listdir():
    try: 
        print('>>', file)
        process_file(file)
    except Exception as e:
        print(">>>>>>>>", file, e)       

print("")
        
os.chdir(processing_folder)
print("New Files{}: ".format(processing_folder), len(os.listdir()), os.listdir())
os.chdir(base_folder)
    

Processing Files in  D:\IDOC\raw\
>> Dec 2010 Stock Pop.xls
>> Dec 2011 Prison Stock Pop.xls
>> Dec 2012_ Prison_Stock_Internet_Variables.xls
>> Dec 2013_Prison_Stock_Internet_Variables.xls
>> Dec 2014_ Prison_Stock_Internet_Variables.xls
>> Dec 2020  Prison Stock.xls
>> Dec 31 2019 Prison Stock pop.xls
>> December 2006 stock pop.xls
>> December 2007 stock pop.xls
>> December 2008 stock pop.xls
>> December 2009 stock pop.xls
>> December 2018_ Prison Stock.xls
>> Internet Data Set Prison_Stock_Dec_2015.xls
>> Internet Data Set Prison_Stock_Dec_2016.xls
>> Internet Data Set Prison_Stock_Dec_2017.xls
>> Internet Data Set Prison_Stock_June 2017.xls
>> June 2005 stock pop.xls
>> June 2006 stock pop.xls
>> June 2007 stock pop.xls
>> June 2008 stock pop.xls
>> June 2009 stock pop.xls
>> June 2010 Stock Pop.xls
>> June 2011 Prison Stock Pop.xls
>> June 2012 Prison_Stock_Internet_Variables.xls
>> June 2013 _ Prison_Stock_Internet_Variables.xls
>> June 2014_ Prison_Stock_Internet_Variables.xls
>

In [3]:

from csv import reader

def prepare_datafile(filepath):
    '''open and read CSV files into Dataset, display header fields'''
    try:
        file = open(filepath, encoding="utf-8")
        dataset = list(reader(file))
        # 5 rows before headers
        header = dataset[5:6][0]
        data = dataset[7:]
        #print('Header Fields ({}):\n{}\n'.format(len(header),header))
        file.close
    except FileNotFoundError:
        print('***********{} not found.\n'.format(filepath))
        return [],[]
    return (header, data)

#prepare_datafile('D:\\IDOC\\clean\\12-2010.csv')


In [4]:

def explore_data(data, start, end):
    '''function for viewing data'''
    print('# of rows: {:,}\n# of columns: {}\n\nExamples:'
          .format(len(data),len(data[0])))
    #List comprehension to print out every row in slice
    [print(row) for row in data[start:end]]
    print('\n')


In [5]:

def remove_corrupt_records(data, header_len):
    '''find problem row/s (# of fields not equal to header fields) and remove them'''
    for row in data:
        if len(row) != header_len:
            idx = data.index(row)
            print('**** Deleting corrupt record at index {}:\n{}'.format(idx,row))
            del data[idx]

In [6]:

def find_duplicates(data, name_idx):
    '''find duplicate records'''
    unique_rows = []
    duplicate_rows = []
    for row in data:
        name = row[name_idx]
        if(name in unique_rows):
            duplicate_rows.append(name)
        else:
            unique_rows.append(name)


## Data Cleaning

##### process files by datestring 
##### keep track of changing header fields

In [9]:


os.chdir(processing_folder)


def build_header_dict(headers):
    header_values_state = {}
    for i in range(0, len(headers)):
            header_values_state[i] = headers[i]
    #header_values_state = [i:headers[i] for i in range(0, len(headers))]
           
    return header_values_state
    
print('Processing Files in', processing_folder)

header_fields = {}
for file in os.listdir():
    try: 
        print('Processing File', file, end="")
        (headers, data) = prepare_datafile(file)
        
        for x_values, y_values in zip(
            build_header_dict(headers).items(), build_header_dict(header_fields).items()):
            print(".", end="")
            if x_values != y_values:
                print("")
                print('>>>>>>>> Header difference found:\n{}\n{}'.format(x_values, y_values))
        header_fields = headers
        print("")
    except Exception as e:
        print(">>>>>>>>", file, e)       

print("")

Processing Files in D:\IDOC\proc\
Processing File 200506.csv
Processing File 200606.csv....................
Processing File 200612.csv....................
Processing File 200706.csv....................
Processing File 200712.csv....................
Processing File 200806.csv....................
Processing File 200812.csv....................
Processing File 200906.csv....................
Processing File 200912.csv....................
Processing File 201006.csv....................
Processing File 201012.csv....................
Processing File 201106.csv...........
>>>>>>>> Header difference found:
(10, 'Projected Mandatory Supervised Release (MSR) Date')
(10, 'Projected Mandatory Supervised Release (MSR) Date2')
.
>>>>>>>> Header difference found:
(11, 'Projected Discharge Date')
(11, 'Projected Discharge Date2')
........
Processing File 201112.csv....................
Processing File 201206.csv...........
>>>>>>>> Header difference found:
(10, 'Projected Mandatory Supervised Release (MSR

In [8]:
os.chdir(base_folder)